In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
from PIL import Image
import torch
import torch.nn as nn
from torch.optim import SGD, Adam
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torchsummary import summary
from torchvision import datasets
import urllib
import torchmetrics
from torchmetrics.classification import MulticlassAUROC
from torchmetrics.classification import MulticlassF1Score

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# modify the fully connected layer of ResNet18
model_res18 = torchvision.models.resnet18(weights = torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
for param in model_res18.parameters():
    param.requires_grad = False
num_ftrs = model_res18.fc.in_features
# add a fc layer with 128 nodes
model_res18.fc = nn.Sequential(
    nn.Linear(num_ftrs, 1000),
    nn.BatchNorm1d(1000), 
    nn.ReLU(), 
    nn.Dropout(0.1),
    nn.Linear(1000, 500),
    nn.BatchNorm1d(500),
    nn.ReLU(),
    nn.Dropout(0.15),
    nn.Linear(500,100),
    nn.BatchNorm1d(100),
    nn.ReLU(),
    nn.Linear(100,4))
model_res18.to(device)

UNet = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)
for name, param in UNet.named_parameters():
    print(name)
for param in UNet.parameters():
    param.requires_grad = False

UNet.conv = nn.Sequential(
                            nn.Conv2d(32,10,2,stride=2),
                            nn.ReLU(),
                            nn.Dropout(0.2),
                            nn.Conv2d(10,1,2,stride=2),
                            nn.ReLU(),
                            nn.Flatten(),
                            nn.Linear(4096,1000),
                            nn.ReLU(),
                            nn.Dropout(0.2),
                            nn.Linear(1000,4))
UNet.cuda()

In [ ]:
def train_batch(x, y, model, opt, loss_fn):
    model.train()
    opt.zero_grad()                    # Flush memory  
    batch_loss = loss_fn(model(x), y)  # Compute loss
    batch_loss.backward()              # Compute gradients
    opt.step()                         # Make a GD step
    
    return batch_loss.detach().cpu().numpy()

In [ ]:
@torch.no_grad()
def accuracy(x, y, model):
  """
  Calculate and return the accuracy. 
  x: features
  y: labels
  Return: accuracy
  """
  model.eval()
  prediction = model(x)
  argmaxes = prediction.argmax(dim=1) # get the predicted label 
  f1 = MulticlassF1Score(num_classes=4,average=None).cuda()
  f1_score = f1(prediction,y)
  auc = MulticlassAUROC(num_classes=4, average=None, thresholds=None).cuda()
  auc_score = auc(prediction,y)
  s = torch.sum((argmaxes == y).float())/len(y)  # calculate test accuracy
  return s.cpu().numpy(),f1_score.cpu().numpy(),auc_score.cpu().numpy()

In [ ]:
def plot(losses,accuracies,n_epochs):
  """
  Plot the training accuracies and losses over number of epochs 
  losses: a list of training loss
  accuracies: a list of training accuracy
  n_epochs: a list of number of epochs
  """
  plt.figure(figsize=(13,3))
  plt.subplot(121)
  plt.title('Training Loss over epochs')
  plt.plot(np.arange(n_epochs) + 1, losses)
  plt.subplot(122)
  plt.title('Training Accuracy over epochs')
  plt.plot(np.arange(n_epochs) + 1, accuracies)

In [ ]:
def plot_f1_auc(f1,auc,n_epochs):
  """
  Plot the training accuracies and losses over number of epochs 
  losses: a list of training loss
  accuracies: a list of training accuracy
  n_epochs: a list of number of epochs
  """
  fig, axs = plt.subplots(2, 2)
  axs[0, 0].plot(np.arange(n_epochs) + 1, f1[...,0])
  axs[0, 0].set_title('Mild Demented')
  axs[0, 1].plot(np.arange(n_epochs) + 1, f1[...,1])
  axs[0, 1].set_title('Moderate Demented')
  axs[1, 0].plot(np.arange(n_epochs) + 1, f1[...,2])
  axs[1, 0].set_title('Non Demented')
  axs[1, 1].plot(np.arange(n_epochs) + 1, f1[...,3])
  axs[1, 1].set_title('Mild Demented')

  for ax in axs.flat:
    ax.set(xlabel='Epoch', ylabel='F1 Score')
  
  for ax in axs.flat:
    ax.label_outer()
  
  fig2, axs2 = plt.subplots(2, 2)
  axs2[0, 0].plot(np.arange(n_epochs) + 1, auc[...,0])
  axs2[0, 0].set_title('Mild Demented')
  axs2[0, 1].plot(np.arange(n_epochs) + 1, auc[...,1])
  axs2[0, 1].set_title('Moderate Demented')
  axs2[1, 0].plot(np.arange(n_epochs) + 1, auc[...,2])
  axs2[1, 0].set_title('Non Demented')
  axs2[1, 1].plot(np.arange(n_epochs) + 1,auc[...,3])
  axs2[1, 1].set_title('Mild Demented')

  for ax in axs2.flat:
    ax.set(xlabel='Epoch', ylabel='AUC')
  
  for ax in axs2.flat:
    ax.label_outer()
  plt.plot()

In [ ]:
def train(trainset, testset, model, name):
  """
  Train the model with training dataset, and return the training losses, training
    accuracies, and number of epochs. 
  trainset: the processed train data with wanted dimensions
  testset:  the processed test data with wanted dimensions
  model: a torchvision model
  name: the name of the model in string
  Return: Testing accuracy, training losses, training accuracies, and number of epochs
  """
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  # load the data
  train_dl = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True,drop_last=True)
  test_dl = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False,drop_last=True)

  # set up loss function and optimization function
  loss_func = nn.CrossEntropyLoss()
  opt = Adam(model.parameters(), lr=1e-3)

  # train and record the loss, accuracies for each training epoch
  losses, accuracies, n_epochs = [], [], 100
  for epoch in range(n_epochs):
    running_vloss = 0.0
    print(f"Running epoch {epoch + 1} of {n_epochs}")
  
    epoch_losses, epoch_accuracies  = [], []
    for batch in train_dl:
      # calculate loss
        x, y = batch
        x, y = x.to(device), y.to(device)
        batch_loss = train_batch(x, y, model, opt, loss_func)
        epoch_losses.append(batch_loss)
    epoch_loss = np.mean(epoch_losses)

    for batch in train_dl:
      # calcualte accuracy 
        x, y = batch
        x, y = x.to(device), y.to(device)
        batch_acc, _, _ = accuracy(x, y, model)
        epoch_accuracies.append(batch_acc)
    
    epoch_accuracy = np.mean(epoch_accuracies)
    
    
    print(f"The training loss and training accuracy is {epoch_loss} and {epoch_accuracy}")
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

  # calculate the testing accuracy using the model we just trained
  epoch_accuracies = []
  batch_num = 0.0
  f1_batch, auc_batch = 0, 0
  for ix, batch in enumerate(test_dl):
      x, y = batch
      x, y = x.to(device), y.to(device)
      batch_acc, batch_f1, batch_auc = accuracy(x, y, model)
      epoch_accuracies.append(batch_acc)
      f1_batch += batch_f1
      auc_batch += batch_auc
      batch_num += 1
  print(f"batch number is {batch_num}")
  f1_batch=f1_batch/batch_num
  auc_batch=auc_batch/batch_num   
  # save the model for future uses
  torch.save(model.state_dict(), name+".pth")
  print(f"Test accuracy: {np.mean(epoch_accuracies)}")
  return np.mean(epoch_accuracies), losses, accuracies, n_epochs, f1_batch, auc_batch

In [ ]:
def main():
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  # transform the images to match the ImageNet image requirementa
  transform = transforms.Compose(
      [transforms.Resize((256,256)),
       transforms.CenterCrop(224),
       transforms.RandomHorizontalFlip(0.4),
       transforms.RandomVerticalFlip(0.2),
       transforms.RandomAffine(40),
       transforms.ToTensor(),
       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
       ]) 
  transform2 = transforms.Compose(
      [transforms.Resize((256,256)),
       transforms.RandomHorizontalFlip(0.4),
       transforms.RandomAffine(180,translate=(0.3,0.3), shear=10),
       transforms.ToTensor()
       ]) 

  # save to data folder
  trainset = torchvision.datasets.ImageFolder(root='data/train',transform=transform)

  testset = torchvision.datasets.ImageFolder(root='data/test',transform=transform)
  # a list of initialized models
  models = [model_res18]
  # the names of the models
  names = ["res_18"]

  test_acc = [] # store the testing accuracy
  for model, name in zip(models,names):
    acc, losses , accuracies, n_epochs, f1_score, auc_score = train(trainset,testset,model, name)
    test_acc.append(acc)
    plot(losses, accuracies, n_epochs)
    print(f"f1 scores is {f1_score}")
    print(f"AUC is {auc_score}")
    plt.show()
  
  print(f"The test accuracy is: {test_acc}")


In [ ]:
if __name__ == "__main__":
  main()

ResNet18 testing accuracy after 100 epoch is 0.6682692.
f1 scores is [0.07342686 0.01508296 0.4341635  0.2828987 ]
AUC is [0.01941244 0.02307692 0.04466501 0.02315964]

In [ ]:
transform = transforms.Compose(
      [transforms.Resize((256,256)),
       transforms.CenterCrop(224),
       transforms.RandomHorizontalFlip(0.4),
       transforms.RandomVerticalFlip(0.2),
       transforms.RandomAffine(40),
       transforms.ToTensor(),
       transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
       ]) 

model_res18.load_state_dict(torch.load("res_18.pth"))
model_res18.eval()
testset = torchvision.datasets.ImageFolder(root='data/test',transform=transform)
test_dl = torch.utils.data.DataLoader(testset, batch_size=1279,shuffle=False,drop_last=True)
f1_result,auc_result = 0,0
batch_num = 0.0
for ix,batch in enumerate(test_dl):
      x,y = batch
      x,y = x.cuda(),y.cuda()
      prediction = model_res18(x)
      f1 = MulticlassF1Score(num_classes=4,average=None).cuda()
      f1_score = f1(prediction,y)
      auc = MulticlassAUROC(num_classes=4, average=None, thresholds=None).cuda()
      auc_score = auc(prediction,y)
      f1_result+=f1_score.cpu().numpy()
      auc_result+=auc_score.cpu().numpy()
      batch_num +=1
print(f"average f1 scores is {f1_result/batch_num}")
print(f"average auc is {auc_result/batch_num}")